In [1]:
import time

start_time = time.time()

### Importações das Bibs Necessárias


In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio import AlignIO
import matplotlib.pyplot as plt
import os
import tempfile

### Leitura da sequência de referência

In [3]:
def read_reference_sequence(reference_file_path):
    with open(reference_file_path) as handle:
        reference_sequence = "".join(line.strip().lower() for line in handle)
    return reference_sequence

### Leitura das Sequências

Recebe o path do arquivo fasta e retorna array com sequências


In [4]:
def read_gene_sequences(gene_file_path):
    gene_sequences = []
    with open(gene_file_path) as handle:
        for record in SeqIO.parse(handle, "fasta"):
            sequence = str(record.seq).lower()
            gene_sequences.append(sequence)
    return gene_sequences

### Alinhamento Múltiplo com Referência

Recebe uma sequência de referência e as sequências genicas e realiza o alinhamento com o ClutalOmega

In [5]:
def alignment_with_reference_per_sequence(reference_sequence, gene_sequence):
    input_file = "input.fasta"
    output_file = "output.fasta"

    # Adiciona a sequência de referência ao arquivo "input.fasta"
    fake_reference = SeqRecord(Seq(reference_sequence), id="reference")
    with open(input_file, "w") as handle:
        SeqIO.write([fake_reference, SeqRecord(Seq(gene_sequence), id="gene")], handle, "fasta")

    # Executa o alinhamento usando o Clustal Omega
    clustalomega_cline = ClustalOmegaCommandline(
        infile=input_file,
        outfile=output_file,
        verbose=True,
        auto=True,
        force=True  # Força a sobrescrita do arquivo de saída se existir
    )
    clustalomega_cline()

    # Remove os arquivos temporários
    os.remove(input_file)

    # Lê o arquivo de saída do alinhamento
    alignment = AlignIO.read(output_file, "fasta")
    os.remove(output_file)

    return alignment

### Visualização dos Resultados

In [6]:
def visualize_alignment(alignment):
    # Calcula o percentual de identidade para cada posição do alinhamento
    identity_percentages = []
    for column in range(alignment.get_alignment_length()):
        column_data = alignment[:, column]
        identity_percentage = column_data.count(column_data[0]) / len(column_data) * 100
        identity_percentages.append(identity_percentage)

    # Plota o gráfico de percentual de identidade
    plt.plot(identity_percentages)
    plt.xlabel("Position")
    plt.ylabel("Identity Percentage")
    plt.title("Identity Percentage by Position")
    plt.show()

### Salva alinhamento em um arquivo

In [7]:
def save_alignment_to_file(alignments, output_file_path):
    # Salva todos os alinhamentos em um único arquivo
    with open(output_file_path, "a") as handle:
        for alignment in alignments:
            for i, record in enumerate(alignment):
                if i != 0:  # Pula a primeira sequência (a de referência)
                    original_header = record.description.split(' ', 1)[1]
                    SeqIO.write(SeqRecord(seq=record.seq, id=record.id, description=original_header), handle, "fasta")

### Main


In [10]:
# Caminho para o arquivo com os genes
gene_file_path = "/home/maume/UnEB/TCC/Dataset/sequencias_teste_bkp/A.2.bkp/gene_spike.fasta"

# Pré-processamento gene referência
#Caminho para o arquivo com a seq de referência
gene_reference_path = "/home/maume/UnEB/TCC/Implementation/refSpike.fasta"

# Lê a sequência de referência
reference_sequence = read_reference_sequence(gene_reference_path)

# Lê as sequências genicas
gene_sequences = read_gene_sequences(gene_file_path)

# Realiza o sequenciamento múltiplo com referência com o Clustal Omega para cada sequência genica
alignments = []
for gene_sequence in gene_sequences:
    alignment = alignment_with_reference_per_sequence(reference_sequence, gene_sequence)
    alignments.append(alignment)

Alignment with 2 rows and 3822 columns
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa reference
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa gene
Alignment with 2 rows and 3822 columns
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa reference
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa gene
Alignment with 2 rows and 3822 columns
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa reference
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa gene
Alignment with 2 rows and 3822 columns
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa reference
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa gene
Alignment with 2 rows and 3822 columns
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa reference
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa gene
Alignment with 2 rows and 3822 columns
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa reference
atgtttgtttttcttgttttattgccactagtctctagtcagtg...taa gene
Alignment with 2 rows and 3822 columns
atgtttgtttttcttgttttattgc

### Salva alinhamentos em arquivo e gera visualizações

In [ ]:
# print(alignments)

# # Caminho para o arquivo de saída com os alinhamentos
# output_file_path = "/home/maume/UnEB/TCC/Implementation/saida.fasta"

# # Salva todos os alinhamentos em um único arquivo
# save_alignment_to_file(alignments, output_file_path)

# Visualiza os resultados do alinhamento
# for alignment in alignments:
#     visualize_alignment(alignment)

# print('Alinhamento Completo Concluído.')

In [11]:
from datetime import datetime

end_time = time.time()

elapsed_time = end_time - start_time
current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

print(elapsed_time)
print(current_datetime)

# for i, alignment in enumerate(alignments):
#             print(f"Alignment {i + 1}:")
#             print(alignment)
#             print("\n")

# for alignment in alignments:
# visualize_alignment(alignments[0])

10130.6289229393
2023-09-12 11:44:51


In [ ]:
# import numpy as np

# # print(alignment.substitutions)

# # Obter a matriz de substituições do alinhamento
# substitutions = alignment.substitutions

# # Lista com as bases representadas na matriz
# bases = ['a', 'c', 'g', 't']

# # Cria o heatmap
# plt.imshow(substitutions, cmap='hot', interpolation='nearest')
# plt.colorbar()

# # Define os rótulos dos eixos x e y com as bases
# plt.xticks(np.arange(len(bases)), bases)
# plt.yticks(np.arange(len(bases)), bases)

# # Exibe o gráfico
# plt.xlabel('Substituição por')
# plt.ylabel('Base')

# plt.title('Matriz de Substituições')

# plt.show()

In [13]:
output_alinhamento = "/home/maume/UnEB/TCC/Dataset/sequencias_teste_bkp/A.2/alinhamento.fasta"
with open(output_alinhamento, "w") as f:
        AlignIO.write(alignments, f, "fasta")

### Salvando Informações Sobre a Execução

In [ ]:
import platform
import psutil

machine_info = platform.uname()
memory_info = psutil.virtual_memory()

path_directory = '/home/maume/UnEB/TCC/Implementation/04_pipeline_alinhamento_multiplo'

content = '============================================================\n'
content += 'Informações Sobre a Última Execução:\n\n'
content += f'- Quantidade de Variantes Analisadas: {files_processed}\n'
content += f'- Quantidade de Genomas Analisados: {sequences_processed}\n'
content += f'- Quantidade de Genomas em que o Gene não foi Encontrado: {genes_not_found}\n'
content += f'- Data e Hora da Execução: {current_datetime}\n'
content += f'- Tempo Total de Execução: {round(elapsed_time, 2)} Segundos\n'
content += f'- Caminho do Dataset Utilizado: {root_directory}\n'
content += f'\nInformações Sobre a Máquina Utilizada\n\n'
content += f'- S.O.: Ubuntu 22.04.2 LTS x86_64\n'
content += f'- Kernel: {machine_info.release}\n'
content += f'- CPU: Intel i7-6500U (4) @ 3.100GHz {machine_info.processor}\n'
content += f'- Total de Memória: {round(memory_info.total / (1024 ** 3), 2)}GB\n'


output_file_name = 'alignment_execution_information.txt'
output_file_path = os.path.join(path_directory, output_file_name)

with open(output_file_path, 'a') as output_file:
    output_file.write(content)

print(f'Information saved to file: {output_file_path}')